## Linking with Spark
pyspark는 driver 와 worker들의 python version 이 동일해야한다.

In [3]:
from pyspark import SparkContext, SparkConf

## Initializing Spark 
### SparkContext
어떻게 cluster 에 접근할지 Spark 에게 알려줌
### SparkConf
내 application 정보 저장  
### SparkSession 와 SparkConf 차이?
#### 공통점
Spark 앱 진입점  
<br>

#### 차이점
SparkSession 은 2.0 부터 도입된 새로운 진입점  
SparkContext 는 RDD API 중심 지원, SparkSession 은 RDD + DataFrame + SQL 등 모든 API 지원  
SparkSession 내부에 SparkContext 포함(spark.sparkContext)

In [4]:
conf = SparkConf().setAppName("RDD Programming Guide").setMaster("local")
sc = SparkContext(conf=conf)

25/04/22 23:36:06 WARN Utils: Your hostname, gimjunhaui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.45.202 instead (on interface en0)
25/04/22 23:36:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/22 23:36:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Using shell

shell 에서 사용법은 다음에   
python driver 설정 내용이 있는데 필요할 때 알아보자

## Resilient Distributed Datasets (RDDs)
RDDs 는 고장에 강하고 병렬로 처리할 수 있는 데이터 요소들의 집합  
RDDs 생성은 드라이버 프로그램에 있는 데이터 요소들 혹은 외부 저장장치에 있는 데이터셋 (HDFS, HBase, Hadoop 기반)

### Parallelized Collection
Parallelized Collection은 드라이버 프로그램에서 기존의 리스트나 반복 가능한 객체에 대해 SparkContext의 parallelize 메소드를 호출함으로 생성  
데이터 요소들은 복사되어 병렬로 처리할 수 있는 RDDs 가 됨

In [23]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
distData

MemoryError: 

일단 생성을 하면 분산된 데이터셋(distData) 는 병렬로 처리 가능  
예를 들어 아래와 같이 코드를 작성하면 모든 요소의 합을 구할 수 있다.

In [18]:
distData.reduce(lambda a, b: a + b)

15

Parallelized Collection 에서 가장 중요한 요소(변수) 하나는 데이터셋을 몇개의 파티션으로 나눌지이다.  
Spark 는 각 클러스터의 파티션에 대해 한가지 task 만 실행  
보통 클러스터의 CPU 당 2~4 파티션으로 나눈다. 
Spark 는 자동으로 클러스터 기반으로 파티션 갯수를 나눈다.  
그러나, 수동으로 몇개의 파티션을 나눌지 설정이 가능

In [19]:
sc.parallelize(data, 10) # 10개의 파티션으로 나눈다

ParallelCollectionRDD[30] at readRDDFromFile at PythonRDD.scala:289

In [2]:
# 실험 시간 갯수
# 1~1000억 하니 컴퓨터 메모리가 못버틴다...
# 메모리 에러 발생 -> SparkConf 기본 메모리가 1G 로 되어 있어서 아래와같이 설정
data1 = list(range(100000000))
conf1 = SparkConf().setAppName("MyApp").set("spark.executor.memory", "2g").set("spark.driver.memory", "2g")
sc1 = SparkContext(conf=conf1)

25/04/18 00:22:21 WARN Utils: Your hostname, gimjunhaui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.45.202 instead (on interface en0)
25/04/18 00:22:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/18 00:22:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
distData1 = sc1.parallelize(data1, 2)
distData1.reduce(lambda a, b: a + b)

25/04/18 00:19:01 WARN TaskSetManager: Stage 0 contains a task of very large size (245355 KiB). The maximum recommended task size is 1000 KiB.


4999999950000000

In [3]:
distData2 = sc1.parallelize(data1, 10)
distData2.reduce(lambda a, b: a + b)

25/04/18 00:22:37 WARN TaskSetManager: Stage 0 contains a task of very large size (48972 KiB). The maximum recommended task size is 1000 KiB.


4999999950000000

#### 2개 파티션 vs 10개 파티션 시간 비교
1~1억 이 들어있는 List 의 sum 을 하나씩 더해서 구하는 경우  
2개 파티션 24.832초  
10개 파티션 17.91초  
나눠서 하면 빠르다!

### External Datasets
PySpark은 Hadoop 에서 호환되는 모든 저장 source로 분산된 데이터셋 생성이 가능 (local file system, HDFS, Cassandra, HBase, AWS S3 등)  
Spark 는 텍스트파일 sequence 파일 혹은 다른 Hadoop 입력형식을 지원한다.  

<strong>textFile</strong> method 을 사용해서 텍스트파일 RDDs 생성이 가능  
이 method 은 파일의 URI(로컬 경로, hdfs://, s3a:// 등의 URI) 를 가지고 줄 단위로 읽는다.  
생성된 RDDs 텍스트파일은 dataset 기능을 사용가능하다.

In [20]:
distFile = sc.textFile("../data/data.txt")
# distFile(data.txt) 의 총 길이
distFile.map(lambda s: len(s)).reduce(lambda a, b: a + b)

999

Spark에서 파일 읽는 경우 참고 사항 (textFile)
* 만약 로컬 파일을 쓰는 경우 노드 내에도 같은 경로에 접근 가능한 파일이 존재해야한다. 모든 노드에 복사 혹은 노드에 공유된 network-mounted 파일 시스템 사용
* 파일을 읽을때 파일경로(sample.txt), 와일드카드(\*.txt 같은), 압축파일(*.gz) 이 사용 가능하다
* textFile에는 두번째 인자가 있는데 이는 파일이 몇개의 파티션으로 구성될지 설정. 기본값으로 파일블럭(블럭은 HDFS 에서 128MB 기본값)당 1개의 파티션 생성, block 갯수 > 파티션 갯수

Spark 파이썬 API 지원하는 데이터 형식
* <strong>SparkContext.wholeTextFiles</strong> 은 경로 안에 있는 작은 여러 텍스트 파일을 읽고, 각각 (파일명, 내용) 쌍으로 리턴함. textFile은 다르게 각파일에서 각줄당 1개의 결과만 리턴
* <strong>RDD.saveAsPickleFile</strong> and <strong>SparkContext.pickleFile</strong>은 RDD를 간단한 형식인 pickle 의 Python object로 저장을 지원, 배치가 pickle serialization 에 사용, 배치 기본 크기는 10.
* <strong>SequenceFile</strong> 와 <strong>Hadoop Input/Output</strong>형식

### Spark SQL 의 read/write 지원은 실험 단계. 고수를 위한거므로 일단은 패스...

### RDD Operations
2가지의 operation 지원 
transformations, actions

 

#### transformations    
기존의 dataset으로 새로운 dataset 생성  

#### actions 
dataset의 실행 결과인 하나의 값을 driver program 으로 전달.

In [ ]:
# 이해를 돕기위한 예
distFile = sc.textFile("../data/data.txt")
distFile.map(lambda s: len(s)).reduce(lambda a, b: a + b)

<code>distFile.map(lambda s: len(s))</code> 은 transformations 에 해당  
각 줄마다 길이를 갖는 새로운 dataset 으로 변환  

<code>distFile.map(lambda s: len(s)).reduce(lambda a, b: a + b)</code> 은 actions 에 해당  
위의 transformation 한 dataset 에 대해서 각줄의 길이의 합(전체 길이) 를 하나의 값으로 반환  
(참고: reduceByKey 는 distributed dataset 반환)

모든 transformations 은 lazy(게으름), 실제로 실행을 바로 하지 않음.  
transformation 해야하는거만 기억해둠.  
action 이 일어나지 않으면 실제 행하지 않음.   
이를 통해 Spark가 더 효율적으로 실행 가능 

기본적으로 transform RDD 는 매번 action 이 일어나면 실행하지만, 자주 사용하는 RDD 를 메모리에 <strong>persist</strong>(cache, 캐싱) 가능.  
자주 쓰이므로 메모리에 올려두고 더 빠르게 접근 가능(매번 transform 하지 않아도 된다)  
추가로 RDD를 디스크에 persisting 하거나 여러 노드에 복제할 수 있다.  

#### Basics

In [ ]:
lines = sc.textFile("../data/data.txt") # 파일 위치를 그저 가르킬 뿐, 메모리에 올리거나 해당 RDD 에 대한 action 이 없으면 행하지 않음
lineLengths = lines.map(lambda s: len(s)) # transformation, 당장 실행하지 않음, 게으름
totalLength = lineLengths.reduce(lambda a, b: a + b) # action, 여러 장치에서 돌리기 위해 task 단위로 할일을 나눔, 각 장치에서 map 과 reduce 실행 지금 실행하는 위치에서만 결과값을 반환 받음
print(totalLength)

# 나중에 lineLengths 를 쓴다면
lineLengths.persist()

#### Passing Functions to Spark
클러스터 내에서 실행하는 Spark API는 driver program 이 보내주는 function 에 의지한다.  
3가지 권장된 방법:  
1. Lambda 표현식(Lambda는 multi-statement function(여러줄로 된 함수)과 return 값이 없는 것을 지원하지 않음
2. spark 연산을 호출하는 함수 내부에 로컬 함수를 정의하는 방식(코드가 좀 길도, 외부에서 안 쓰는 함수)
3. 함수 정의를 스크림트나 모듈의 최상단에 위치시키는 것 (재사용 시, 여러 spark 작업에서 쓸 때)

In [ ]:
# 1. lambda 표현식
rdd = sc.textFile("../data/data.txt")
rdd.map(lambda s: len(s)).reduce(lambda a, b: a + b)

In [16]:
# 2. 로컬 함수 사용
def process_data():
    def is_even(x):
        return x % 2 == 0
    
    rdd = sc.parallelize(range(10))
    even_rdd = rdd.filter(is_even)
    print(even_rdd.collect())
process_data()

[0, 2, 4, 6, 8]


In [17]:
# 3. 최상단 함수
def is_even(x):
    return x % 2 == 0

rdd = sc.parallelize(range(10))
even_rdd = rdd.filter(is_even)
print(even_rdd.collect())

[0, 2, 4, 6, 8]


In [23]:
# 최상단
def myFunc(s):
    words = s.split(" ")
    return len(words)

sc.textFile("../data/data.txt").map(myFunc).collect()

[4,
 5,
 1,
 4,
 1,
 2,
 1,
 1,
 6,
 3,
 4,
 13,
 8,
 6,
 3,
 10,
 7,
 3,
 7,
 2,
 25,
 1,
 1,
 1,
 1,
 3,
 1,
 5,
 4,
 36,
 2,
 8,
 5,
 1,
 1,
 1,
 1]

In [22]:
# 클래스를 이용해서 다른 함수 사용 가능
class MyClass(object):
    def func(self, s):
        return len(s.split(' '))
    def doStuff(self, rdd):
        return rdd.map(self.func)
mc = MyClass()
mc.doStuff(sc.textFile("../data/data.txt")).collect()

[4,
 5,
 1,
 4,
 1,
 2,
 1,
 1,
 6,
 3,
 4,
 13,
 8,
 6,
 3,
 10,
 7,
 3,
 7,
 2,
 25,
 1,
 1,
 1,
 1,
 3,
 1,
 5,
 4,
 36,
 2,
 8,
 5,
 1,
 1,
 1,
 1]